<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://meteofrance.fr/" ><img src="photo.jpg" style="float:right; max-width: 120px; display: inline"  alt="MétéoFrance"/></a>


</center>

# Projet Recherche et Innovation, par Eva Marques, Aleksi Timonen, et Solange Pruilh
# Tuteur : M. Philippe Besse

# Deuxième Partie : 
## B- Prévision de la nébulosité (Méteo-France )  en <a href="https://www.python.org/"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Python_logo_and_wordmark.svg/390px-Python_logo_and_wordmark.svg.png" style="max-width: 120px; display: inline" alt="Python"/></a> avec <a href="http://keras.io"><img src="keras.png" style="max-width: 70px; display: inline" alt="Keras"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import utils
import os
%matplotlib inline

path=""
data=pd.read_csv(path+"data.dat",sep=",",header=0)

In [3]:

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,SimpleRNN, Lambda,GRU,AveragePooling1D
from keras.layers import Conv2D, MaxPooling2D,LSTM,LocallyConnected2D,Convolution2D,Reshape,Conv1D
from keras.utils import np_utils
from keras.optimizers import RMSprop,Nadam,Adam,SGD
from keras import backend as K

print(keras.__version__)

Using TensorFlow backend.


2.0.4


In [4]:
data

,nebsat,lat,lon,flevap3SOL0,flir3SOL0,fllat3SOL0,huH2,nbSOL0,nhSOL0,nmSOL0,ntSOL0,ntSOL0em3,ntSOL0ep3,pMER0,tH2,tH2VGRAD,ffH1000,ddfH1000,mois,ep_opt
0,5.44,41.0,-6.0,-0.051682,-43808,-129392,97.087520,7.4375,0.0000,0.0000,8.0000,7.8125,7.4375,1024.1106,-4.734543,5.142542,4.284757,N,1,2873.60700
1,8.00,41.0,-5.5,-0.036545,-39200,-90480,97.331661,8.0000,0.0000,0.0000,8.0000,6.2500,8.0000,1024.0506,-4.750168,5.033167,4.655296,N,1,4821.00400
2,8.00,41.0,-5.0,-0.027267,-39712,-67952,96.623653,7.9375,0.0000,0.0000,8.0000,7.3750,7.9375,1024.1706,-4.343918,4.330042,3.927262,N,1,5560.81000
3,8.00,41.0,-4.5,-0.027267,-38688,-68976,97.697872,7.6875,0.0000,0.0000,8.0000,8.0000,7.6875,1023.9506,-4.703293,4.126917,6.208558,N,1,4501.77400
4,8.00,41.0,-4.0,-0.030685,-21280,-77168,96.770138,7.9375,0.0000,0.0000,8.0000,8.0000,7.9375,1023.4606,-6.234543,3.486292,7.838200,N,1,5900.37600
5,6.40,41.0,-3.5,-0.288010,-700704,-722288,85.368770,1.0000,0.0000,0.0000,2.6875,2.7500,1.0000,1021.3806,-5.218918,4.361292,5.034113,N,1,129.65480
6,8.00,41.0,-3.0,-0.151291,-826144,-379248,83.366817,0.0000,0.0000,0.0000,0.4375,0.1250,0.0000,1021.3206,-7.093918,6.892542,8.350292,N,1,0.00000
7,8.00,41.0,-2.5,-0.116135,-30496,-289136,95.207638,8.0000,0.1875,0.5000,8.0000,8.0000,8.0000,1022.1506,-4.390793,3.345667,9.241476,N,1,2973.02400
8,8.00,41.0,-2.0,-0.103439,-28448,-258416,94.133419,8.0000,0.3750,0.6250,8.0000,6.3750,8.0000,1021.7306,-4.343918,3.033167,9.732744,N,1,3221.44700
9,8.00,41.0,-1.5,-0.112228,-198944,-280944,87.126583,7.2500,1.6875,1.5625,7.0000,6.0625,7.3750,1020.8106,-6.031418,4.361292,14.575819,N,1,2217.39100


In [5]:
data["ddfH1000"]=pd.Categorical(data["ddfH1000"], ordered=False)
data["mois"]=pd.Categorical(data["mois"], ordered=False)

del data["fllat3SOL0"]



In [6]:
from math import sqrt,log
data["Sq_huH2"]=data["huH2"].map(lambda x: x**2)
data["sqrt_ffH1000"]=data["ffH1000"].map(lambda x: sqrt(x))
data["sq_tH2"]=data["tH2"].map(lambda x : x**2)
data["sq_tH2VGRAD"]=data["tH2VGRAD"].map(lambda x : x**2)
data["nbSOL0:ep_opt"]=data["nbSOL0"]*data["ep_opt"]
del data["huH2"]
del data["ffH1000"]

data["nebsat"]=data["nebsat"].map(lambda x: 100*x/8)
data["ntSOL0"]=data["ntSOL0"].map(lambda x: 100*x/8)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564740 entries, 0 to 564739
Data columns (total 22 columns):
nebsat           564740 non-null float64
lat              564740 non-null float64
lon              564740 non-null float64
flevap3SOL0      564740 non-null float64
flir3SOL0        564740 non-null int64
nbSOL0           564740 non-null float64
nhSOL0           564740 non-null float64
nmSOL0           564740 non-null float64
ntSOL0           564740 non-null float64
ntSOL0em3        564740 non-null float64
ntSOL0ep3        564740 non-null float64
pMER0            564740 non-null float64
tH2              564740 non-null float64
tH2VGRAD         564740 non-null float64
ddfH1000         564740 non-null category
mois             564740 non-null category
ep_opt           564740 non-null float64
Sq_huH2          564740 non-null float64
sqrt_ffH1000     564740 non-null float64
sq_tH2           564740 non-null float64
sq_tH2VGRAD      564740 non-null float64
nbSOL0:ep_opt    564740 non-

In [31]:
data["lat"]

0         41.0
1         41.0
2         41.0
3         41.0
4         41.0
5         41.0
6         41.0
7         41.0
8         41.0
9         41.0
10        41.0
11        41.0
12        41.0
13        41.0
14        41.0
15        41.0
16        41.0
17        41.0
18        41.0
19        41.0
20        41.0
21        41.0
22        41.0
23        41.0
24        41.0
25        41.0
26        41.0
27        41.0
28        41.0
29        41.0
          ... 
564710    51.5
564711    51.5
564712    51.5
564713    51.5
564714    51.5
564715    51.5
564716    51.5
564717    51.5
564718    51.5
564719    51.5
564720    51.5
564721    51.5
564722    51.5
564723    51.5
564724    51.5
564725    51.5
564726    51.5
564727    51.5
564728    51.5
564729    51.5
564730    51.5
564731    51.5
564732    51.5
564733    51.5
564734    51.5
564735    51.5
564736    51.5
564737    51.5
564738    51.5
564739    51.5
Name: lat, Length: 564740, dtype: float64

In [7]:
def classes(x):
    y=x
    y[(y<=10) & (y>0)]=2
    y[(y>10) & (y<=25)]=3
    y[(y>25) & (y<=50)]=4
    y[(y>50) & (y<=75)]=5
    y[y>75]=6
    y[y==0]=1
    return y

In [8]:
def one_bloc(nebsat, lat,lon):
    #On extraie le bloc 7x7 sur lequel on va travailler
    nebsat7x7=nebsat[(nebsat["lat"]==lat) | (nebsat["lat"]==lat+0.5) | (nebsat["lat"]==lat+1) | (nebsat["lat"]==lat+1.5) | (nebsat["lat"]==lat-0.5) | (nebsat["lat"]==lat-1) | (nebsat["lat"]==lat-1.5)]
    nebsat7x7=nebsat7x7[(nebsat7x7["lon"]==lon) | (nebsat7x7["lon"]==lon+0.5) | (nebsat7x7["lon"]==lon+1) | (nebsat7x7["lon"]==lon+1.5) | (nebsat7x7["lon"]==lon-0.5) | (nebsat7x7["lon"]==lon-1) | (nebsat7x7["lon"]==lon-1.5)]
    return nebsat7x7

In [9]:
new_data=data
#new_data=one_bloc(data,45.0,0.0)
print (new_data.shape)
new_data.info()

(564740, 22)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564740 entries, 0 to 564739
Data columns (total 22 columns):
nebsat           564740 non-null float64
lat              564740 non-null float64
lon              564740 non-null float64
flevap3SOL0      564740 non-null float64
flir3SOL0        564740 non-null int64
nbSOL0           564740 non-null float64
nhSOL0           564740 non-null float64
nmSOL0           564740 non-null float64
ntSOL0           564740 non-null float64
ntSOL0em3        564740 non-null float64
ntSOL0ep3        564740 non-null float64
pMER0            564740 non-null float64
tH2              564740 non-null float64
tH2VGRAD         564740 non-null float64
ddfH1000         564740 non-null category
mois             564740 non-null category
ep_opt           564740 non-null float64
Sq_huH2          564740 non-null float64
sqrt_ffH1000     564740 non-null float64
sq_tH2           564740 non-null float64
sq_tH2VGRAD      564740 non-null float64
nbSOL0:ep_opt  

In [10]:
neb=new_data["nebsat"][ : :]
nebC=classes(neb)
new_data["nebsatC"]=nebC
del new_data["nebsat"]
nebsat=new_data.iloc[:,21]
new_data.drop(new_data.columns[[21]], axis=1,inplace=True)
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564740 entries, 0 to 564739
Data columns (total 21 columns):
lat              564740 non-null float64
lon              564740 non-null float64
flevap3SOL0      564740 non-null float64
flir3SOL0        564740 non-null int64
nbSOL0           564740 non-null float64
nhSOL0           564740 non-null float64
nmSOL0           564740 non-null float64
ntSOL0           564740 non-null float64
ntSOL0em3        564740 non-null float64
ntSOL0ep3        564740 non-null float64
pMER0            564740 non-null float64
tH2              564740 non-null float64
tH2VGRAD         564740 non-null float64
ddfH1000         564740 non-null category
mois             564740 non-null category
ep_opt           564740 non-null float64
Sq_huH2          564740 non-null float64
sqrt_ffH1000     564740 non-null float64
sq_tH2           564740 non-null float64
sq_tH2VGRAD      564740 non-null float64
nbSOL0:ep_opt    564740 non-null float64
dtypes: category(2), float64

In [11]:
nebsatDum=pd.get_dummies(new_data[["mois","ddfH1000"]])
nebsatQuant=new_data[["lat","lon","flevap3SOL0","flir3SOL0","Sq_huH2","nbSOL0","nhSOL0","nmSOL0","ntSOL0","ntSOL0em3","ntSOL0ep3","pMER0","tH2","tH2VGRAD","sqrt_ffH1000","ep_opt","sq_tH2","sq_tH2VGRAD","nbSOL0:ep_opt"]]
new_data=pd.concat([nebsatDum,nebsatQuant],axis=1)

In [12]:
new_data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564740 entries, 0 to 564739
Data columns (total 36 columns):
mois_1           564740 non-null uint8
mois_2           564740 non-null uint8
mois_3           564740 non-null uint8
mois_4           564740 non-null uint8
mois_5           564740 non-null uint8
mois_6           564740 non-null uint8
mois_7           564740 non-null uint8
mois_8           564740 non-null uint8
mois_9           564740 non-null uint8
mois_10          564740 non-null uint8
mois_11          564740 non-null uint8
mois_12          564740 non-null uint8
ddfH1000_C       564740 non-null uint8
ddfH1000_E       564740 non-null uint8
ddfH1000_N       564740 non-null uint8
ddfH1000_S       564740 non-null uint8
ddfH1000_W       564740 non-null uint8
lat              564740 non-null float64
lon              564740 non-null float64
flevap3SOL0      564740 non-null float64
flir3SOL0        564740 non-null int64
Sq_huH2          564740 non-null float64
nbSOL0           564740

In [13]:
print (nebsat.shape)
print (new_data.shape)

(564740,)
(564740, 36)


In [14]:
Matrix=pd.DataFrame.as_matrix(new_data)
#Test=pd.DataFrame.as_matrix(nebsat)

In [15]:
print (Matrix.shape)
var=36
grille=748
days=755
sample=748
kernel_size=(7,1)
batch_size=128
mat3D=np.reshape(Matrix,(grille,days,var))
mat3Dtest=np.reshape(nebsat,(grille,days,1))
#print (mat3Dtest)

(564740, 36)


/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


In [16]:
X_train=mat3D[:sample,:700,:]
X_test=mat3D[:sample,700:,:]
input_shape=(sample,1,var)
Y_train=mat3Dtest[:sample,:700]
Y_test=mat3Dtest[:sample,700:]

X_train=X_train.reshape(X_train.shape[1],sample,1,var)
X_test=X_test.reshape(X_test.shape[1],sample,1,var)
Y_train=Y_train.reshape(Y_train.shape[1],sample,1)
Y_test=Y_test.reshape(Y_test.shape[1],sample,1)
#print (Y_train)
print (Y_train.shape)
print (X_train.shape)


(700, 748, 1)
(700, 748, 1, 36)


# 1- Premier réseau de neurones

In [1]:
print (input_shape)
model=Sequential()

model.add(Activation('relu',input_shape=input_shape))
model.add(Conv2D(10,(7,1),border_mode='same'))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(sample))

model.add(Reshape((sample,1)))

model.add(SimpleRNN(sample))
model.add(Activation('hard_sigmoid'))

model.add(Reshape((sample,1,1)))
model.add(Conv2D(1,(7,1),border_mode='same'))
model.add(Reshape((sample,1)))
model.add(Activation('softmax'))
model.add(Lambda(lambda x:x*5+1))


   

NameError: name 'input_shape' is not defined

In [ ]:
model.summary()

In [ ]:
from keras import optimizers
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss',min_delta=0, patience=2)

model.compile(loss='mean_squared_error', optimizer=Adam(),metrics=['accuracy'])

In [ ]:

#callbacks=[early_stopping]
model.fit(X_train,Y_train,batch_size=batch_size,epochs=10,verbose=1,validation_data=(X_test,Y_test))

Train on 700 samples, validate on 55 samples
Epoch 1/10
128/700 [====>.........................] - ETA: 178s - loss: 6.5906 - acc: 0.5100

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

On teste avec un autre optimizer.

In [21]:
model.reset_states
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
activation_1 (Activation)    (None, 748, 1, 34)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 748, 1, 34)        8126      
_________________________________________________________________
dropout_1 (Dropout)          (None, 748, 1, 34)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 25432)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 748)               19023884  
_________________________________________________________________
reshape_1 (Reshape)          (None, 748, 1)            0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 748)               561000    
__________

In [22]:
model.compile(loss='mean_squared_error', optimizer=RMSprop(),metrics=['accuracy'])
batch_size=128
#callbacks=[early_stopping]
model.fit(X_train,Y_train,batch_size=batch_size,epochs=10,verbose=1,validation_data=(X_test,Y_test))

Train on 700 samples, validate on 55 samples
Epoch 1/10
700/700 [==============================] - 59s - loss: 6.9863 - acc: 0.4980 - val_loss: 6.9734 - val_acc: 0.5102
Epoch 2/10
700/700 [==============================] - 58s - loss: 6.9863 - acc: 0.4980 - val_loss: 6.9734 - val_acc: 0.5102
Epoch 3/10
700/700 [==============================] - 59s - loss: 6.9863 - acc: 0.4980 - val_loss: 6.9734 - val_acc: 0.5102
Epoch 4/10
700/700 [==============================] - 59s - loss: 6.9863 - acc: 0.4980 - val_loss: 6.9734 - val_acc: 0.5102
Epoch 5/10
700/700 [==============================] - 59s - loss: 6.9863 - acc: 0.4980 - val_loss: 6.9734 - val_acc: 0.5102
Epoch 6/10
700/700 [==============================] - 60s - loss: 6.9863 - acc: 0.4980 - val_loss: 6.9734 - val_acc: 0.5102
Epoch 7/10
700/700 [==============================] - 59s - loss: 6.9863 - acc: 0.4980 - val_loss: 6.9734 - val_acc: 0.5102
Epoch 8/10
700/700 [==============================] - 59s - loss: 6.9863 - acc: 0.4980 

In [23]:
score=model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 6.97335922935
Test accuracy: 0.510184747523


## 2- Deuxième réseau de neurones

In [17]:
model = Sequential()
model.add(Activation('relu',input_shape=input_shape))

model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(sample))

model.add(Reshape((sample,1)))
model.add(AveragePooling1D(pool_size=7,padding='same'))
model.add(SimpleRNN(sample))
model.add(Activation('hard_sigmoid'))


model.add(Reshape((sample,1)))
model.add(Activation('softmax'))
model.add(Lambda(lambda x:x*5+1))

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
activation_1 (Activation)    (None, 748, 1, 36)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 748, 1, 36)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 26928)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 748)               20142892  
_________________________________________________________________
reshape_1 (Reshape)          (None, 748, 1)            0         
_________________________________________________________________
average_pooling1d_1 (Average (None, 107, 1)            0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 748)               561000    
__________

In [19]:
model.compile(loss='mean_squared_error', optimizer=RMSprop(),metrics=['accuracy'])
batch_size=128
#callbacks=[early_stopping]
model.fit(X_train,Y_train,batch_size=batch_size,epochs=10,verbose=1,validation_data=(X_test,Y_test))

Train on 700 samples, validate on 55 samples
Epoch 1/10
700/700 [==============================] - 12s - loss: 6.9863 - acc: 0.4980 - val_loss: 6.9734 - val_acc: 0.5102
Epoch 2/10
700/700 [==============================] - 10s - loss: 6.9863 - acc: 0.4980 - val_loss: 6.9734 - val_acc: 0.5102
Epoch 3/10
700/700 [==============================] - 10s - loss: 6.9863 - acc: 0.4980 - val_loss: 6.9734 - val_acc: 0.5102
Epoch 4/10
700/700 [==============================] - 10s - loss: 6.9863 - acc: 0.4980 - val_loss: 6.9734 - val_acc: 0.5102
Epoch 5/10
700/700 [==============================] - 10s - loss: 6.9863 - acc: 0.4980 - val_loss: 6.9734 - val_acc: 0.5102
Epoch 6/10
700/700 [==============================] - 10s - loss: 6.9863 - acc: 0.4980 - val_loss: 6.9734 - val_acc: 0.5102
Epoch 7/10
700/700 [==============================] - 10s - loss: 6.9863 - acc: 0.4980 - val_loss: 6.9734 - val_acc: 0.5102
Epoch 8/10
700/700 [==============================] - 10s - loss: 6.9863 - acc: 0.4980 

In [20]:
score=model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 6.97335922935
Test accuracy: 0.510184747523
